### References

https://medium.com/bioquest/pyseal-homomorphic-encryption-in-a-user-friendly-python-package-51dd6cb0411c

# Imports

In [1]:
# General python libraries
import numpy as np
import matplotlib as plt

# PySEAL imports - taken from SEALPythonExamples/examples.py
import time
import random
import threading
import seal
from seal import ChooserEvaluator,     \
                 Ciphertext,           \
                 Decryptor,            \
                 Encryptor,            \
                 EncryptionParameters, \
                 Evaluator,            \
                 IntegerEncoder,       \
                 FractionalEncoder,    \
                 KeyGenerator,         \
                 MemoryPoolHandle,     \
                 Plaintext,            \
                 SEALContext,          \
                 EvaluationKeys,       \
                 GaloisKeys,           \
                 PolyCRTBuilder,       \
                 ChooserEncoder,       \
                 ChooserEvaluator,     \
                 ChooserPoly


General PySEAL parameters - see http://130.56.248.129:8889/edit/SEALPythonExamples/examples.py . A high modulus slows computation down, but seems necessary to generate correct results.

In [2]:
# General params
parms = EncryptionParameters()
parms.set_poly_modulus("1x^8192 + 1")
parms.set_coeff_modulus(seal.coeff_modulus_128(8192))
parms.set_plain_modulus(786433)
context = SEALContext(parms)

Set up the secret and public keys.

In [3]:
keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()

Create encryption, evaluator, decryption and encoder objects.

In [4]:
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)
encoder = FractionalEncoder(context.plain_modulus(), context.poly_modulus(), 64, 32, 3)

In [5]:
class Cloud:
    def __init__(self, context, public_key):
        self.context = context
        self.encryptor = Encryptor(self.context, public_key)
        self.evaluator = Evaluator(self.context)
        self.encoder = FractionalEncoder(self.context.plain_modulus(), self.context.poly_modulus(), 64, 32, 3)
        self.encrypted_data = np.array([])
        
    def submit_to_the_cloud(self, encrypted_data):
        self.encrypted_data = np.append(self.encrypted_data, encrypted_data)
    
    def submit_to_the_cloud_and_encrypt(self, plain_data):
        encrypted_data = []
        for i in range(len(plain_data)):
            encrypted_data.append(Ciphertext(self.context.parms()))
            self.encryptor.encrypt(self.encoder.encode(plain_data[i]), encrypted_data[i])
        self.submit_to_the_cloud(encrypted_data)
    
    def zero(self):
        value = 0;
        plain = self.encoder.encode(value)
        encrypted_zero = Ciphertext(self.context.parms())
        self.encryptor.encrypt(plain, encrypted_zero)
        return encrypted_zero
        
    def mean(self):
        encrypted_result = self.zero()
        data = np.ndarray.flatten(np.array(self.encrypted_data))
        self.evaluator.add_many(data, encrypted_result)
        divisor = self.encoder.encode(1 / len(data))
        self.evaluator.multiply_plain(encrypted_result, divisor)
        return encrypted_result
    
    def variance(self):
        mu = self.mean()
        data = np.ndarray.flatten(np.array(self.encrypted_data))
        encrypted_result = self.zero()
        variances = []
        for i in range(len(data)):
            d = data[i]
            variances.append(self.zero())
            res = variances[i]
            self.evaluator.add(res, d)
            self.evaluator.negate(res)
            self.evaluator.add(res, mu)
            self.evaluator.square(res)
        encrypted_result = Ciphertext()
        evaluator.add_many(variances, encrypted_result)
        divisor = self.encoder.encode(1 / len(data))
        evaluator.multiply_plain(encrypted_result, divisor)
        return encrypted_result
            
    # "At risk" means mean + 2 x sigma, i.e. ~top 5 percentile
    # Convoluted calculation, due to no square root, division or comparison with zero
    # Once decrypted, > 0 means at risk
    def at_risk(self, encrypted_value):
        mu = self.mean()
        evaluator.negate(mu)
        sigma = self.variance()
        evaluator.multiply_plain(sigma, self.encoder.encode(4)) 
        evaluator.negate(sigma)

        evaluator.add(encrypted_value, mu)
        evaluator.square(encrypted_value)
        evaluator.add(encrypted_value, sigma)
        return encrypted_value


Blood pressure values. Taken from https://thl.fi/publications/monica/bp/table8.htm

In [6]:
mu = 131
sigma = 18
blood_pressure = sigma * np.random.randn(10) + mu



Encrypt blood pressure values

In [7]:
encrypted_rationals = []
for i in range(len(blood_pressure)):
    encrypted_rationals.append(Ciphertext(parms))
    encryptor.encrypt(encoder.encode(blood_pressure[i]), encrypted_rationals[i])

Check non-encrypted values.

In [8]:
print("Unencrypted mean: " + (str)(np.mean(blood_pressure))[:8])
print("Unencrypted variance: " + (str)(np.var(blood_pressure))[:8])
print("Unencrypted std. dev.: " + (str)(np.std(blood_pressure))[:8])


Unencrypted mean: 132.9285
Unencrypted variance: 267.4493
Unencrypted std. dev.: 16.35387


Simple test of our "Cloud". Tests that a zero value is created.

In [9]:
cloud = Cloud(context, public_key)
cloud.zero()

In [10]:
cloud = Cloud(context, public_key)
z = cloud.zero()
plain_result = Plaintext()
decryptor.decrypt(z, plain_result)
result = encoder.decode(plain_result)
print("Zero: " + (str)(result)[:8])


Zero: 0.0


Create a "Cloud", submit some data, and obtain the mean and variance.

In [11]:
cloud = Cloud(context, public_key)
cloud.submit_to_the_cloud(encrypted_rationals)
mu = cloud.mean()
variance = cloud.variance()



Output cloud-generated values.

In [12]:

plain_result = Plaintext()
decryptor.decrypt(mu, plain_result)
result = encoder.decode(plain_result)
print("Blood pressure average: " + (str)(result)[:8])
plain_result = Plaintext()
decryptor.decrypt(variance, plain_result)
result = encoder.decode(plain_result)
print("Blood pressure variance: " + (str)(result)[:8])
print("Blood pressure std. dev.: " + (str)(np.sqrt(result))[:8])


Blood pressure average: 132.9285
Blood pressure variance: 267.4493
Blood pressure std. dev.: 16.35387


Create a test patient value to compare.

In [17]:
patient_X_bp = 160
encrypted_patient_value = Ciphertext(parms)
encryptor.encrypt(encoder.encode(patient_X_bp), encrypted_patient_value)

In [18]:
sq_mu = np.mean(blood_pressure)
print(sq_mu)
sq_mu = np.var(blood_pressure)
print(sq_mu * 4)
print(np.square(patient_X_bp - np.mean(blood_pressure)))

132.92851180752135
1069.7972026453433
732.8654729555112


Check whether the patient is "at risk".

In [23]:
ar = cloud.at_risk(encrypted_patient_value)
plain_result = Plaintext()
decryptor.decrypt(ar, plain_result)
result = encoder.decode(plain_result)
print("Blood pressure at risk: " + (str)(result > 0))

Blood pressure at risk: False


In [21]:
cloud = Cloud(context, public_key)
cloud.submit_to_the_cloud_and_encrypt(blood_pressure)
encrypted_result = cloud.mean()

In [22]:
plain_result = Plaintext()
decryptor.decrypt(encrypted_result, plain_result)
result = encoder.decode(plain_result)
print("Blood pressure average: " + (str)(result)[:8])

Blood pressure average: 132.9285
